In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from binascii import unhexlify
from pathlib import Path
from common import *

DATASET_PATH = "../tpmscan-dataset"

def load_key_measurements(dir):
    results = []

    for path in Path(dir).rglob("*/detail/Keygen:ECC_*.csv"):
        path = str(path)

        measurement = path.split(os.sep)[-3]
        curve = CURVES_REVERSED[int(path.split(os.sep)[-1].split("Keygen:ECC_")[1][:-4], 16)]

        try:
            data = pd.read_csv(path, sep="[,;]", engine="python")
        except:
            print(f"Error reading file: {path}")
            continue

        secrets = data["private"]
        timings = data["duration"]
        if data[["private", "duration"]].isnull().values.any():
            print(f"Failed collection: {path}")
            continue

        secrets = map(unhexlify, secrets)

        results.append({
            "name": measurement,
            "info": get_device_info(Path(path).parent.parent / "results.yaml"),
            "path": path,
            "curve": curve,
            "secrets": list(secrets),
            "timings": list(timings),
        })

    return results

measurements = load_key_measurements(DATASET_PATH)


In [ ]:
# Save concatenated keys to keys/ directory
os.makedirs("keys", exist_ok=True)
for m in measurements:
    with open(f"keys/{m['name']}_{m['curve']}.bin", "wb") as f:
        f.write(b"".join(m["secrets"]))


In [ ]:
# Save timing plots to keys_plot/ directory
os.makedirs("keys_plots", exist_ok=True)
for m in measurements:
    plt.figure(figsize=(10, 10))
    plt.title(f"{m['info']['manufacturer']} {m['info']['firmware']} {m['info']['revision']} {m['curve']}")
    plt.xlabel("Key MSB")
    plt.ylabel("Time (s)")
    plt.xscale("log", base=2)
    plt.xticks([2 ** x for x in range(0, 9)])
    plt.grid(True, axis="x")

    points = zip(list(map(lambda x: int(x[0]), m["secrets"])), m["timings"])
    points = sorted(points, key=lambda x: x[1])[:-5]
    points = zip(*points)

    plt.scatter(*points, marker=".", color="#d81b60")
    plt.savefig(f"keys_plots/{m['info']['manufacturer']}_{m['info']['firmware']}_{m['curve']}_{m['name']}.png", dpi=300, bbox_inches="tight")
    plt.close()
    break # remove to generate all plots
